# Registration example using multiscale-imaging
This notebook contains an example of applying the registration with the functions transferred into the multiscale-imaging package.

In [1]:
from pathlib import Path
import numpy as np
import SimpleITK as sitk
%matplotlib notebook
import matplotlib.pyplot as plt
import math
import pandas as pd
import tiffile as tif
import pickle
import os

import multiscale.microscopy.ome as ome
import multiscale.LINK_system.coordinate as coord
import multiscale.itk.transform as tran
import multiscale.itk.itk_plotting as iplt
import multiscale.ultrasound.reconstruction as recon

# Set up registration
First define the paths to each needed file.  This includes the transform parameters, the images themselves, as well as the US position list and the first tile of the microscopy image.

In [2]:
fiducial_dir = Path(r'F:\Research\LINK\Eye images\Fiducial')
sample_dir = Path(r'F:\Research\LINK\Eye images\2019-10-11 - Eye v2\Eye Images')

registered_dir = Path(r'F:\Research\LINK\Eye images\2019-10-11 - Eye v2\Eye Images\Registered')

Load the SHG image

In [94]:
microscopy_path = Path(sample_dir, 'SHG _ down4x.tif')
microscopy_tile_path = Path(sample_dir, 'SHG settings.ome.tif')
microscopy_image = coord.open_microscopy(microscopy_path, microscopy_tile_path, downsample_factor=4)

Load the US image

In [41]:
microscopy_image.GetSpacing()

(4.08, 4.08, 20.0)

In [34]:
us_path = Path(sample_dir, 'US Z3225_Run-1.tif')
us_pl_path = Path(sample_dir, 'US Z3225.pos')
us_param_path = Path(sample_dir, 'US Z3225_Run-1_Settings.mat')

us_transform_path = Path(fiducial_dir, 'L22 Transform.txt')

spacing = [36, 36, 36]
dynamic_range = 80
gauge_height = 3225
us_image = coord.open_us(us_path, us_pl_path, us_param_path, spacing, gauge_height, dynamic_range)

Load the OCT image

In [109]:
oct_path = Path(sample_dir, 'OCT _ second image _ 60dB8bit.tif')
oct_pl_path = Path(sample_dir, 'OCT Position v2.pos')
spacing = [2.5, 2.5, 7.8]
zero_reference = 3211.33
oct_image = coord.open_oct(oct_path, oct_pl_path, spacing, zero_reference)

oct_transform_path = Path(r'F:\Research\LINK\System calibration\2019-10-04 - 4x and 20x fiducial', 'OCT to SHG.txt')

In [ ]:
oct_im

Load the coordinate transform

In [104]:
with open(us_transform_path, 'rb') as fp:
    us_coordinate_transform = pickle.load(fp)
    
with open(oct_transform_path, 'rb') as fp:
    oct_coordinate_transform = pickle.load(fp)

Register SHG to US and save the results

In [105]:
us_coordinate_transform[5] = us_coordinate_transform[5] - 520

In [36]:
shg_reg_path = Path(registered_dir, 'SHG to US.tif')

In [49]:
shg_reg = tran.apply_transform_params(us_image, microscopy_image, 
                                     -1*us_coordinate_transform,
                                     sitk.VersorRigid3DTransform())

In [50]:
sitk.WriteImage(sitk.Cast(shg_reg, sitk.sitkUInt8), str(shg_reg_path))

In [40]:
us_rotated_path = Path(registered_dir, 'US rotated.tif')
sitk.WriteImage(sitk.Cast(us_image, sitk.sitkUInt8), str(us_rotated_path))

Register the OCT to SHG

In [85]:
oct_coordinate_transform[3] = oct_coordinate_transform[3] + 200
oct_coordinate_transform[4] = oct_coordinate_transform[4] + 300

In [110]:
oct_reg = tran.apply_transform_params(microscopy_image, oct_image, oct_coordinate_transform, sitk.VersorRigid3DTransform())

In [91]:
oct_reg_path = Path(registered_dir, 'OCT to SHG.tif')

In [114]:
sitk.WriteImage(oct_reg, str(oct_reg_path))

Register the OCT to US

In [111]:
oct_to_us = tran.apply_transform_params(us_image, oct_reg,
                                     -1*us_coordinate_transform,
                                     sitk.VersorRigid3DTransform())

In [112]:
oct_us_path = Path(registered_dir, 'OCT to US.tif')

In [113]:
sitk.WriteImage(sitk.Cast(oct_to_us, sitk.sitkUInt8), str(oct_us_path))